# Coleta automatica - Template2

In [3]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 6.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.0/359.0 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 7.6 MB/s eta 0:00:00


Com o objetivo de iniciar um estudo acerca do reaproveitamento de coletores dentro de um mesmo template, esse notebook discute uma abordagem inicial a ser realizada no Template 2. Para isso, utiliza-se as seguintes bibliotecas:\
-BeaufifulSoup para gerar links \
-Selenium e Playwright para realização dos processamentos dinâmicos\
Casos isoladas como baixar pdfs que abrem em nova guia são tratados usando a ferramenta de automação PyAutogui.

## Bibliotecas

In [67]:
import os
import time
import urllib3
import pyautogui
import asyncio
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException, ElementNotInteractableException

from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
from urllib.parse import urljoin

A seguir, encontram-se as cidades e as respectivas urls bases. Também há as subtags a serem coletadas. Essas subtags são adaptadas para serem usadas no casamento de padrão com as urls das páginas.

In [71]:
cidades = ['Novo Oriente de Minas',
     'Frei Gaspar',
     'Fronteira dos Vales',
     'Chapada do Norte',
     'Sardoá',
     'Bertópolis',
     'Cuparaque',
     'Coroaci',
     'Machacalis',
     'Periquito',
     'Itabirinha',
     'São Sebastião do Maranhão',
     'Caraí',
     'Ataléia',
     'Ouro Verde de Minas',
     'Itaipé',
     'Umburatiba',
     'Marilac',
     'Jordânia',
     'São João Evangelista',
     'Bandeira',
     'Tumiritinga',
     'Santa Efigênia de Minas',
     'Pavão',
     'Pescador',
     'Cantagalo',
     'Santo Antônio do Jacinto']

tags = ["licitacoes", "contratos", "Empenhos", "Receitas", "folha", "Pagamentos", 
        "liquidacoes","diarias", "obras", "Relatorio", "dispensas", "despesas-com-pessoal", "editais"]
subpastas = ["screenshots", "files", "htmls"]

cidades_urls = {'Novo Oriente de Minas': ["https://novoorientedeminas.mg.gov.br/transparencia", "https://novoorientedeminas.mg.gov.br/tran"],
     'Frei Gaspar': ["https://freigaspar.mg.gov.br/transparencia", "https://freigaspar.mg.gov.br/tran"],
     'Fronteira dos Vales': ["https://fronteiradosvales.mg.gov.br/transparencia", "https://fronteiradosvales.mg.gov.br/tran"],
     'Chapada do Norte': ["https://chapadadonorte.mg.gov.br/transparencia", "https://chapadadonorte.mg.gov.br/tran"],
     'Sardoá': ["https://sardoa.mg.gov.br/transparencia", "https://sardoa.mg.gov.br/tran"],
     'Bertópolis': ["https://bertopolis.mg.gov.br/transparencia", "https://bertopolis.mg.gov.br/tran"],
     'Cuparaque': ["https://cuparaque.mg.gov.br/transparencia", "https://cuparaque.mg.gov.br/tran"],
     'Coroaci': ["https://coroaci.mg.gov.br/transparencia", "https://coroaci.mg.gov.br/tran"],
     'Machacalis': ["https://machacalis.mg.gov.br/transparencia", "https://machacalis.mg.gov.br/tran"],
      #'Periquito':
     'Itabirinha': ["https://itabirinha.mg.gov.br/transparencia", "https://itabirinha.mg.gov.br/tran"],
     'São Sebastião do Maranhão': ["https://saosebastiaodomaranhao.mg.gov.br/transparencia", "https://saosebastiaodomaranhao.mg.gov.br/tran"],
     'Caraí': ["https://carai.mg.gov.br/transparencia", "https://carai.mg.gov.br/tran"],
     'Ataléia': ["https://ataleia.mg.gov.br/transparencia", "https://ataleia.mg.gov.br/tran"],
     'Ouro Verde de Minas': ["https://ouroverdedeminas.mg.gov.br/transparencia", "https://ouroverdedeminas.mg.gov.br/tran"],
     'Itaipé': ["https://itaipe.mg.gov.br/transparencia", "https://itaipe.mg.gov.br/tran"],
     'Umburatiba': ["https://umburatiba.mg.gov.br/transparencia", "https://umburatiba.mg.gov.br/tran"],
     'Marilac': ["https://marilac.mg.gov.br/transparencia", "https://marilac.mg.gov.br/tran"],
     'Jordânia': ["https://jordania.mg.gov.br/transparencia", "https://jordania.mg.gov.br/tran"],
     'São João Evangelista': ["https://sje.mg.gov.br/transparencia", "https://sje.mg.gov.br/tran"],
     'Bandeira': ["https://bandeira.mg.gov.br/transparencia", "https://bandeira.mg.gov.br/tran"],
     'Tumiritinga': ["https://tumiritinga.mg.gov.br/transparencia", "https://tumiritinga.mg.gov.br/tran"],
     'Santa Efigênia de Minas': ["https://santaefigenia.mg.gov.br/transparencia", "https://santaefigenia.mg.gov.br/tran"],
     'Pavão': ["https://pavao.mg.gov.br/transparencia", "https://pavao.mg.gov.br/tran"],
     'Pescador': ["https://pescador.mg.gov.br/transparencia", "https://pescador.mg.gov.br/tran"],
     'Cantagalo': ["https://cantagalo.mg.gov.br/transparencia", "https://cantagalo.mg.gov.br/tran"],
     'Santo Antônio do Jacinto': ["https://www.santoantoniodojacinto.mg.gov.br/transparencia", "https://www.santoantoniodojacinto.mg.gov.br/tran"]}

pasta_raiz = os.getcwd()
pasta_template = "/Coletas" + "/template2/"
pasta_template_completo = pasta_raiz + pasta_template

In [72]:
print(pasta_template)

/Coletas/template2/


## Funções

A função a seguir cria as pastas para guardar adequadamente cada um dos dados coletados

In [73]:
def create_pastes():

    try:
        os.mkdir(pasta_raiz + "/Coletas")
    except FileExistsError:
        pass
    
    try:
        os.mkdir(pasta_raiz + "/Coletas" + "/template2/")
    except FileExistsError:
        pass
    
    for cidade in cidades:
        try:
            os.mkdir(pasta_template_completo + cidade)
            for tag in tags:
                os.mkdir(pasta_template_completo  + cidade + "/" + tag)
                for sub in subpastas:
                    os.mkdir(pasta_template_completo  + cidade + "/" + tag + "/" + sub)
        except FileExistsError:
            continue
    
create_pastes()

### Coletor 1

Esse primeiro coletor é específico para baixar urls que representam pdfs. Assim, utiliza-se o Selenium para abrir a url e o PyAutogui para realizar o download.

In [93]:
def coletor_pdf(cidade, subtag, url):

    chrome_options = Options()
    #options.add_argument('--headless')
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_experimental_option('prefs', {
    "download.default_directory": pasta_template_completo + cidade + "/"+ subtag + "/files",#IMPORTANT - ENDING SLASH V IMPORTANT, #Change default directory for downloads
    "download.prompt_for_download": False, #To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
    })

    driver = webdriver.Chrome(options = chrome_options)
    driver.get(url)

    time.sleep(2)
    pyautogui.hotkey('ctrl', 'b')
    time.sleep(2)
    pyautogui.press('enter')
    driver.close()
    

Teste unitário

In [94]:
coletor_pdf("Frei Gaspar", "relatorios", "https://digitaliza-institucional.s3.us-east-2.amazonaws.com/municipio-de-cantagalo/rgf_rreo/Relat%C3%B3rio%20Resumido%20de%20Execu%C3%A7%C3%A3o%20Or%C3%A7ament%C3%A1ria%20Simplificado%20-%201%C2%BA%20bimestre-06-07-2021%20-%20nwsBT.pdf")

### Coletor 2

In [85]:
async def coletor_geral(cidade, subtag, url): 

    try:
    
        playwright = await async_playwright().start()
        browser = await playwright.chromium.launch(headless = False)
        page = await browser.new_page()

        await page.goto(url)

        async with page.expect_download() as download_info:
            await page.locator("button:has-text(\"CSV\")").click()
            #await page.locator("#DataTables_Table_0_wrapper button:has-text(\"CSV\")").click()
            #await page.locator("#DataTables_Table_1_wrapper button:has-text(\"CSV\")").click()
        download = await download_info.value

        filename = await download.path()
        complement = str(filename).split("/")[-1]
        
        await page.screenshot(path=pasta_template + cidade + "/" + subtag + "/" + 
                              "screenshots/" + "page"+complement+".png")
        await download.save_as(pasta_template + cidade + "/" + subtag + "/files/" + complement)
        
    except:
        
        playwright_2 = await async_playwright().start()
        browser_2 = await playwright.chromium.launch(headless = False)
        page_2 = await browser.new_page()
        
        await page_2.goto(url)
        conteudo = await page_2.content()
        
        url_aux = url.replace(".", "")
        print(pasta_template+cidade+"/"+subtag+"/"+"htmls/"+url_auxl + ".html")
        with open(pasta_template+cidade+"/"+subtag+"/"+"htmls/"+url_auxl + ".html", "w") as file:
            file.write(conteudo)
        file.close()
        
        await page_2.close()
        await browser_2.close()
        
    finally:
        
        await page.close()
        await browser.close()
        
        
        

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(coletor_html("Novo Oriente de Minas", "Empenhos", "https://novoorientedeminas.mg.gov.br/transparencia/liquidacoes/exibir/2022/06/81808"))

### Coletor 3

In [82]:
async def coletor_html(cidade, subtag, url): 
        
    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(headless = False)
    page = await browser.new_page()
    await page.goto(url)
    conteudo = await page.content()
    
    url_aux = url.replace(".", "")
    url_aux = url_aux.replace("/", "")
    url_aux = url_aux.replace(":", "")
    subtag = subtag.lower()
    
    with open(pasta_template_completo+cidade+"/"+subtag+"/"+"htmls/"+url_aux + ".html", "w") as file:
        file.write(conteudo)
    file.close()
        
    await page.close()
    await browser.close()

Teste unitário

In [83]:
loop = asyncio.get_event_loop()
loop.create_task(coletor_html("Novo Oriente de Minas", "Empenhos", "https://novoorientedeminas.mg.gov.br/transparencia/liquidacoes/exibir/2022/06/81808"))

<Task pending name='Task-108' coro=<coletor_html() running at /tmp/ipykernel_4953/2045983809.py:1>>

httpsnovoorientedeminasmggovbrtransparencialiquidacoesexibir20220681808
/Coletas/template2/Novo Oriente de Minas/empenhos/htmls/httpsnovoorientedeminasmggovbrtransparencialiquidacoesexibir20220681808.html


### Explorador de links

É nessa função que iremos gerar os links correspondentes a cada subtag para cada um dos municípios considerados. Esses links são separados e retornados em um dicionário.

In [84]:
def crawl(pages, depth, tags, restriction):
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    already_visited = set(pages)
    
    dic = {}
    for tag in tags:
        dic[tag] = set()

    for i in range(depth):
        new_pages = set()
        print("Actual depth: " + str(i))

        for page in pages:
            http = urllib3.PoolManager()
            try:
                page_data = http.request('GET', page)
            except:
                print("Error: " + page)
                continue


            soup = BeautifulSoup(page_data.data, "lxml")
            already_visited.add(page)

            links = soup.find_all('a')
            
            counter = 0
            for link in links:
                counter = 0
                if("href" in link.attrs):
                    url = urljoin(page, str(link.get('href')))

                    if url.find("'") != -1:
                        continue

                    url = url.split("#")[0]

                #Restriction to prevent the crawler get out the base url
                if url.startswith(restriction) or url.startswith("https://digitaliza"):
                    if "exibir" not in url:
                        new_pages.add(url)
                    for tag in tags:
                        if tag in url:
                            dic[tag].add(url)
                            counter+=1
                            #if tag == "obras":
                             #   dic[tag].add(url)
                            #elif "detalhes" in url:
                             #   dic[tag].add(url)
                    if "pdf" in url:
                        dic["Relatorio"].add(url)
            
                
                

        pages = new_pages.difference(already_visited)
        print(pages)

        print("Number of links visited: " +  str(len(already_visited)))

    return dic

### Coleta

A função a seguir junta todos os passos anteriores. Assim, ela recebe uma cidade, sua url bases e as restrições de geração de links e, para cada link gerado, realiza a coleta do dado.

In [31]:
def colect(cidade, url, restriction, loop):
    
    links = crawl([url], 5, tags, restriction)
    
    for key, value in links.items():
        for link in value:
            if "pdf" in link:
                coletor_pdf(cidade, key, link)
            else:
                loop.create_task(coletor_unico(cidade, key, link))
    
    print("Relatorio de coleta da cidade " + "/" + cidade + "/")
    for subtag in os.listdir(pasta_template+ cidade):
        print("Coletado " + str(len(os.listdir(pasta_template + cidade + "/" + subtag + "/files"))) + 
              " documentos na subtag " + subtag)

Teste unitário


### Execução em massa

In [9]:
loop = asyncio.get_event_loop()
for key, value in cidades_urls.items():
    colect(key, value[0], value[1], loop)

Actual depth: 0
{'https://novoorientedeminas.mg.gov.br/transparencia/empenhos', 'https://novoorientedeminas.mg.gov.br/transparencia/pagamentos', 'https://novoorientedeminas.mg.gov.br/transparencia/coronavirus', 'https://novoorientedeminas.mg.gov.br/transparencia/diarias', 'https://novoorientedeminas.mg.gov.br/transparencia/relatorios', 'https://novoorientedeminas.mg.gov.br/transparencia/mapa', 'https://novoorientedeminas.mg.gov.br/transparencia/folhas-de-pagamento', 'https://novoorientedeminas.mg.gov.br/transparencia/dispensas', 'https://novoorientedeminas.mg.gov.br/transparencia/receitas', 'https://novoorientedeminas.mg.gov.br/transparencia/despesas-com-pessoal', 'https://novoorientedeminas.mg.gov.br/transparencia/contratos', 'https://novoorientedeminas.mg.gov.br/transparencia/liquidacoes', 'https://novoorientedeminas.mg.gov.br/transparencia/sobre'}
Number of links visited: 1
Actual depth: 1
{'https://novoorientedeminas.mg.gov.br/transparencia/receitas/detalhes/2021/05/197', 'https://


KeyboardInterrupt



### Resultados

In [9]:
def gerarRelatorio(cidade, url, restriction):
    
    print("Relatorio de coleta da cidade " + "--" + cidade + "--")
    for subtag in os.listdir(pasta_template+ cidade):
        print("Coletado " + str(len(os.listdir(pasta_template + cidade + "/" + subtag + "/files"))) + 
              " documentos na subtag " + subtag)
        global total
        total += len(os.listdir(pasta_template + cidade + "/" + subtag + "/files"))
    print("\n")

In [15]:
total = 0
for key, value in cidades_urls.items():
    gerarRelatorio(key, value[0], value[1])
print("Total de documentos coletados: " + str(total))

Relatorio de coleta da cidade --Novo Oriente de Minas--
Coletado 1 documentos na subtag receitas
Coletado 1 documentos na subtag relatorios
Coletado 0 documentos na subtag folhas-de-pagamento
Coletado 102 documentos na subtag pagamentos
Coletado 94 documentos na subtag licitacoes
Coletado 108 documentos na subtag diarias
Coletado 0 documentos na subtag empenhos
Coletado 45 documentos na subtag contratos
Coletado 0 documentos na subtag obras


Relatorio de coleta da cidade --Frei Gaspar--
Coletado 0 documentos na subtag receitas
Coletado 0 documentos na subtag relatorios
Coletado 0 documentos na subtag folhas-de-pagamento
Coletado 0 documentos na subtag pagamentos
Coletado 0 documentos na subtag licitacoes
Coletado 0 documentos na subtag diarias
Coletado 0 documentos na subtag empenhos
Coletado 0 documentos na subtag contratos
Coletado 0 documentos na subtag obras


Relatorio de coleta da cidade --Fronteira dos Vales--
Coletado 1 documentos na subtag receitas
Coletado 1 documentos na su